In [1]:
import timm
import torchsummary
import glob
import time
import logging
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
from collections import OrderedDict
from timm.data import Dataset, create_loader, resolve_data_config,  FastCollateMixup, mixup_batch, AugMixDataset
from timm.models import create_model, resume_checkpoint, convert_splitbn_model, apply_test_time_pool
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler
from munch import Munch
import yaml
import sys
from generator import Generator
from datetime import datetime
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
from apex import amp
from apex.parallel import DistributedDataParallel as DDP
from apex.parallel import convert_syncbn_model
has_apex = True

In [3]:
# GPU Device
gpu_id = '0,1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device " , use_cuda)

GPU device  True


In [4]:
torch.backends.cudnn.benchmark = True

In [5]:
with open('config/train.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
args = Munch(config)
args.prefetcher = not args.no_prefetcher
args.distributed = False
args.device = 'cuda'
args.world_size = 1
args.rank = 0
logging.info('Training with a single process on %d GPUs.' % args.num_gpu)

INFO:root:Training with a single process on 3 GPUs.


In [6]:
torch.manual_seed(args.seed + args.rank)

In [7]:
model_ns = timm.create_model('tf_efficientnet_b7_ns', pretrained=True)
model_ns = model_ns.cuda()
# model_ns = torch.nn.DataParallel(model_ns).cuda()

In [8]:
model_raw = timm.create_model('tf_efficientnet_b7', pretrained=True)
model_raw = model_raw.cuda()
# model = torch.nn.DataParallel(model).cuda()

In [9]:
model = Generator((3, 600, 600), args)
model = model.cuda()

In [10]:
train_dir = '/home/data/imagenet/train'
val_dir = '/home/data/imagenet/val'
data_config = resolve_data_config(vars(args), model=model, verbose=args.local_rank == 0)

INFO:root:Data processing configuration for current model + dataset:
INFO:root:	input_size: (3, 600, 600)
INFO:root:	interpolation: bicubic
INFO:root:	mean: (0.485, 0.456, 0.406)
INFO:root:	std: (0.229, 0.224, 0.225)
INFO:root:	crop_pct: 0.875


In [11]:
num_aug_splits = 0

In [12]:
# if args.split_bn:
#     assert num_aug_splits > 1 or args.resplit
#     model = convert_splitbn_model(model, max(num_aug_splits, 2))

In [13]:
optimizer = create_optimizer(args, model)

In [14]:
use_amp = False
if has_apex and args.amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
    use_amp = True
if args.local_rank == 0:
    logging.info('NVIDIA APEX {}. AMP {}.'.format(
        'installed' if has_apex else 'not installed', 'on' if use_amp else 'off'))

INFO:root:NVIDIA APEX installed. AMP on.


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [15]:
if args.distributed:
    if args.sync_bn:
        assert not args.split_bn
        try:
            if has_apex:
                model = convert_syncbn_model(model)
            else:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            if args.local_rank == 0:
                logging.info(
                    'Converted model to use Synchronized BatchNorm. WARNING: You may have issues if using '
                    'zero initialized BN layers (enabled by default for ResNets) while sync-bn enabled.')
        except Exception as e:
            logging.error('Failed to enable Synchronized BatchNorm. Install Apex or Torch >= 1.1')
    if has_apex:
        model = DDP(model, delay_allreduce=True)
    else:
        if args.local_rank == 0:
            logging.info("Using torch DistributedDataParallel. Install NVIDIA Apex for Apex DDP.")
        model = DDP(model, device_ids=[args.local_rank])  # can use device str in Torch >= 1.1
    # NOTE: EMA model does not need to be wrapped by DDP


In [16]:
lr_scheduler, num_epochs = create_scheduler(args, optimizer)

In [17]:
resume_state = {}
resume_epoch = None
start_epoch = 0
if args.start_epoch is not None:
    # a specified start_epoch will always override the resume epoch
    start_epoch = args.start_epoch
elif resume_epoch is not None:
    start_epoch = resume_epoch
if lr_scheduler is not None and start_epoch > 0:
    lr_scheduler.step(start_epoch)

In [18]:
if args.local_rank == 0:
    logging.info('Scheduled epochs: {}'.format(num_epochs))

INFO:root:Scheduled epochs: 200


In [19]:
train_dataset = Dataset(train_dir)
val_dataset = Dataset(val_dir, load_bytes=False, class_map='')

In [20]:
param_count = sum([m.numel() for m in model.parameters()])
logging.info('Model created, param count: %d' % (param_count))

INFO:root:Model created, param count: 70208448


In [21]:
collate_fn = None
if args.prefetcher and args.mixup > 0:
    assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
    collate_fn = FastCollateMixup(args.mixup, args.smoothing, args.num_classes)

In [22]:
model_raw, test_time_pool = apply_test_time_pool(model_raw, data_config, args)
model_raw = torch.nn.DataParallel(model_raw).cuda()
model_ns = torch.nn.DataParallel(model_ns).cuda()

In [23]:
train_loader = create_loader(
        train_dataset,
        input_size=data_config['input_size'],
        batch_size=args.batch_size,
        is_training=True,
        use_prefetcher=args.prefetcher,
        re_prob=args.reprob,
        re_mode=args.remode,
        re_count=args.recount,
        re_split=args.resplit,
        color_jitter=args.color_jitter,
        auto_augment=args.aa,
        num_aug_splits=num_aug_splits,
        interpolation=args.train_interpolation,
        mean=data_config['mean'],
        std=data_config['std'],
        num_workers=args.workers,
        distributed=args.distributed,
        collate_fn=collate_fn,
        pin_memory=args.pin_mem,
        use_multi_epochs_loader=args.use_multi_epochs_loader
    )

In [24]:
crop_pct = 1.0 if test_time_pool else data_config['crop_pct']
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=args.batch_size,
    is_training=False,
    use_prefetcher=args.prefetcher,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=args.workers,
    crop_pct=crop_pct,
    pin_memory=args.pin_mem,
    tf_preprocessing=args.tf_preprocessing)

In [25]:
if args.jsd:
    assert num_aug_splits > 1  # JSD only valid with aug splits set
    train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing).cuda()
    validate_loss_fn = nn.CrossEntropyLoss().cuda()
elif args.mixup > 0.:
    # smoothing is handled with mixup label transform
    train_loss_fn = SoftTargetCrossEntropy().cuda()
    validate_loss_fn = nn.CrossEntropyLoss().cuda()
elif args.smoothing:
    train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing).cuda()
    validate_loss_fn = nn.CrossEntropyLoss().cuda()
else:
    train_loss_fn = nn.CrossEntropyLoss().cuda()
    validate_loss_fn = train_loss_fn

loss_l1_fn = nn.L1Loss().cuda()

In [26]:
eval_metric = args.eval_metric
best_metric = None
best_epoch = None
saver = None
output_dir = ''
if args.local_rank == 0:
    output_base = args.output if args.output else './output'
    exp_name = '-'.join([
        datetime.now().strftime("%Y%m%d-%H%M%S"),
        args.model,
        str(data_config['input_size'][-1])
    ])
    output_dir = get_outdir(output_base, 'train', exp_name)
    decreasing = True if eval_metric == 'loss' else False
    saver = CheckpointSaver(checkpoint_dir=output_dir, decreasing=decreasing)

In [27]:
model_ema = None
if args.model_ema:
    # Important to create EMA model after cuda(), DP wrapper, and AMP but before SyncBN and DDP wrapper
    model_ema = ModelEma(
        model,
        decay=args.model_ema_decay,
        device='cpu' if args.model_ema_force_cpu else '',
        resume=args.resume)

In [28]:
def train_epoch(epoch, model, model_ns, model_raw, loader, optimizer, loss_fn, loss_traj_fn, args,
               lr_scheduler=None, saver=None, output_dir='', use_amp=False, model_ema=None):

    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()
    losses_l1 = AverageMeter()
    losses_traj = AverageMeter()
    
    model.train()
    model_ns.eval()
    model_raw.eval()
    
    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
    for batch_idx, (inputs, target) in enumerate(loader):
        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)
        if not args.prefetcher:
            inputs, target = inputs.cuda(), target.cuda()
        
        z_out = model(z)
        inputs_z = inputs + z_out
        inputs_z = Variable(inputs_z.data, requires_grad=True)
        
        output, traj_raw = model_raw(inputs_z)
        with torch.no_grad():
            output_ns, traj_ns = model_ns(inputs)
        
        loss_ns = loss_fn(output_ns, target)
        loss_raw = loss_fn(output, target)
        loss_l1 = torch.abs(loss_raw - loss_ns)
        
        loss_traj = 0
        for i in range(len(traj_raw)):
            value = loss_traj_fn(traj_raw[i], traj_ns[i])        
            loss_traj += value
        
        loss = args.lambda_l1 * loss_l1 + args.lambda_traj * loss_traj
        
        if not args.distributed:
            losses_l1.update(loss_l1.item(), inputs.size(0))
            losses_traj.update(loss_traj.item(), inputs.size(0))
            losses_m.update(loss.item(), inputs.size(0))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        torch.cuda.synchronize()
        if model_ema is not None:
            model_ema.update(model)
        num_updates += 1

        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            if args.distributed:
                reduced_loss = reduce_tensor(loss.data, args.world_size)
                losses_m.update(reduced_loss.item(), inputs.size(0))

            if args.local_rank == 0:
                logging.info(
                    'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                    'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
                    'Loss_l1: {loss_l1.val:>9.6f} ({loss_l1.avg:>6.4f})  '
                    'Loss_traj: {loss_traj.val:>9.6f} ({loss_traj.avg:>6.4f})  '
                    'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                    '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'LR: {lr:.3e}  '
                    'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                        epoch,
                        batch_idx, len(loader),
                        100. * batch_idx / last_idx,
                        loss=losses_m,
                        loss_l1=losses_l1,
                        loss_traj=losses_traj,
                        batch_time=batch_time_m,
                        rate=inputs.size(0) * args.world_size / batch_time_m.val,
                        rate_avg=inputs.size(0) * args.world_size / batch_time_m.avg,
                        lr=lr,
                        data_time=data_time_m))

                if args.save_images and output_dir:
                    torchvision.utils.save_image(
                        inputs_z,
                        os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                        padding=0,
                        normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(
                model, optimizer, args, epoch, model_ema=model_ema, use_amp=use_amp, batch_idx=batch_idx)

        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)

        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()

    return OrderedDict([('loss', losses_m.avg)])

In [29]:
def val_epoch(model_raw, model, val_loader, criterion, args):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model_raw.eval()
    model.eval()
    with torch.no_grad():
        # warmup, reduce variability of first batch time, especially for comparing torchscript vs non
        inputs = torch.randn((args.batch_size,) + data_config['input_size']).cuda()
        z = Variable(torch.Tensor(np.random.normal(0, 1, (inputs.size(0), args.latent_dim)))).cuda()
        z_out = model(z)
        inputs_z = inputs + z_out
        
        model_raw(inputs_z)
        end = time.time()
        for i, (inputs, target) in enumerate(val_loader):
            if args.no_prefetcher:
                target = target.cuda()
                inputs = inputs.cuda()

            # synthesizing input + generator

            # compute output
            output, foward_list = model_raw(inputs_z)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc1.item(), inputs.size(0))
            top5.update(acc5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.log_freq == 0:
                logging.info(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@1: {top1.val:>7.3f} ({top1.avg:>7.3f})  '
                    'Acc@5: {top5.val:>7.3f} ({top5.avg:>7.3f})'.format(
                        i, len(val_loader), batch_time=batch_time,
                        rate_avg=inputs.size(0) / batch_time.avg,
                        loss=losses, top1=top1, top5=top5))

    results = OrderedDict(
        top1=round(top1.avg, 4), top1_err=round(100 - top1.avg, 4),
        top5=round(top5.avg, 4), top5_err=round(100 - top5.avg, 4),
        param_count=round(param_count / 1e6, 2),
        img_size=data_config['input_size'][-1],
        cropt_pct=crop_pct,
        interpolation=data_config['interpolation'])

    logging.info(' * Acc@1 {:.3f} ({:.3f}) Acc@5 {:.3f} ({:.3f})'.format(
       results['top1'], results['top1_err'], results['top5'], results['top5_err']))
    return results

In [30]:
for epoch in range(start_epoch, num_epochs):
    if args.distributed:
        loader_train.sampler.set_epoch(epoch)

    train_metrics = train_epoch(
        epoch, model, model_ns, model_raw, train_loader, optimizer, train_loss_fn, loss_l1_fn, args,
        lr_scheduler=lr_scheduler, saver=saver, output_dir=output_dir,
        use_amp=use_amp, model_ema=model_ema)

    if (epoch+1)%5 == 0:
        eval_metrics = val_epoch(model_raw, model, val_loader, validate_loss_fn, args)

    if model_ema is not None and not args.model_ema_force_cpu:
        if args.distributed and args.dist_bn in ('broadcast', 'reduce'):
            distribute_bn(model_ema, args.world_size, args.dist_bn == 'reduce')

        ema_eval_metrics = validate(
            model_ema.ema, loader_eval, validate_loss_fn, args, log_suffix=' (EMA)')
        eval_metrics = ema_eval_metrics

    if lr_scheduler is not None:
        # step LR for next epoch
        lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])

    if (epoch+1)%10 == 0:
        update_summary(
            epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
            write_header=best_metric is None)

    if saver is not None:
    # save proper checkpoint with eval metric
        save_metric = eval_metrics[eval_metric]
        best_metric, best_epoch = saver.save_checkpoint(
            model, optimizer, args,
            epoch=epoch, model_ema=model_ema, metric=save_metric, use_amp=use_amp)

INFO:root:Train: 0 [   0/40036 (  0%)]  Loss: 68.870277 (68.8703)  Loss_l1:  0.024983 (0.0250)  Loss_traj: 68.745361 (68.7454)  Time: 19.191s,    1.67/s  (19.191s,    1.67/s)  LR: 1.000e-04  Data: 1.343 (1.343)
INFO:root:Train: 0 [  50/40036 (  0%)]  Loss: 71.447937 (70.4086)  Loss_l1:  0.174618 (0.1011)  Loss_traj: 70.574844 (69.9030)  Time: 1.250s,   25.59/s  (1.638s,   19.54/s)  LR: 1.000e-04  Data: 0.014 (0.040)
INFO:root:Train: 0 [ 100/40036 (  0%)]  Loss: 69.644737 (70.3646)  Loss_l1:  0.105872 (0.0924)  Loss_traj: 69.115372 (69.9025)  Time: 1.223s,   26.17/s  (1.465s,   21.85/s)  LR: 1.000e-04  Data: 0.015 (0.028)
INFO:root:Train: 0 [ 150/40036 (  0%)]  Loss: 70.725746 (70.3518)  Loss_l1:  0.009817 (0.0946)  Loss_traj: 70.676659 (69.8787)  Time: 1.301s,   24.60/s  (1.405s,   22.77/s)  LR: 1.000e-04  Data: 0.014 (0.023)
INFO:root:Train: 0 [ 200/40036 (  0%)]  Loss: 70.423164 (70.3899)  Loss_l1:  0.053527 (0.0930)  Loss_traj: 70.155525 (69.9251)  Time: 1.249s,   25.62/s  (1.374s, 

INFO:root:Train: 0 [2000/40036 (  5%)]  Loss: 70.980766 (70.4256)  Loss_l1:  0.071028 (0.1025)  Loss_traj: 70.625626 (69.9130)  Time: 1.375s,   23.28/s  (1.294s,   24.73/s)  LR: 1.000e-04  Data: 0.015 (0.015)
INFO:root:Train: 0 [2050/40036 (  5%)]  Loss: 69.863304 (70.4288)  Loss_l1:  0.037090 (0.1024)  Loss_traj: 69.677849 (69.9167)  Time: 1.222s,   26.18/s  (1.294s,   24.73/s)  LR: 1.000e-04  Data: 0.014 (0.015)
INFO:root:Train: 0 [2100/40036 (  5%)]  Loss: 70.304855 (70.4262)  Loss_l1:  0.295635 (0.1026)  Loss_traj: 68.826683 (69.9134)  Time: 1.347s,   23.76/s  (1.294s,   24.74/s)  LR: 1.000e-04  Data: 0.014 (0.015)
INFO:root:Train: 0 [2150/40036 (  5%)]  Loss: 70.636070 (70.4240)  Loss_l1:  0.037179 (0.1028)  Loss_traj: 70.450172 (69.9101)  Time: 1.379s,   23.21/s  (1.293s,   24.74/s)  LR: 1.000e-04  Data: 0.015 (0.015)
INFO:root:Train: 0 [2200/40036 (  5%)]  Loss: 71.165314 (70.4226)  Loss_l1:  0.097237 (0.1027)  Loss_traj: 70.679131 (69.9089)  Time: 1.244s,   25.73/s  (1.293s,   

INFO:root:Train: 0 [4000/40036 ( 10%)]  Loss: 68.898170 (70.4463)  Loss_l1:  0.062003 (0.1025)  Loss_traj: 68.588158 (69.9339)  Time: 1.293s,   24.75/s  (1.289s,   24.82/s)  LR: 1.000e-04  Data: 0.013 (0.014)
INFO:root:Train: 0 [4050/40036 ( 10%)]  Loss: 68.857903 (70.4467)  Loss_l1:  0.051381 (0.1025)  Loss_traj: 68.600998 (69.9340)  Time: 1.239s,   25.84/s  (1.289s,   24.82/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [4100/40036 ( 10%)]  Loss: 70.472527 (70.4484)  Loss_l1:  0.022101 (0.1027)  Loss_traj: 70.362022 (69.9348)  Time: 1.380s,   23.19/s  (1.289s,   24.82/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [4150/40036 ( 10%)]  Loss: 70.573898 (70.4512)  Loss_l1:  0.033234 (0.1025)  Loss_traj: 70.407730 (69.9385)  Time: 1.249s,   25.62/s  (1.289s,   24.82/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [4200/40036 ( 10%)]  Loss: 69.866699 (70.4522)  Loss_l1:  0.276013 (0.1027)  Loss_traj: 68.486633 (69.9387)  Time: 1.342s,   23.84/s  (1.289s,   

INFO:root:Train: 0 [6000/40036 ( 15%)]  Loss: 69.926247 (70.4542)  Loss_l1:  0.067334 (0.1035)  Loss_traj: 69.589577 (69.9366)  Time: 1.282s,   24.97/s  (1.288s,   24.85/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [6050/40036 ( 15%)]  Loss: 68.964691 (70.4547)  Loss_l1:  0.028242 (0.1034)  Loss_traj: 68.823479 (69.9376)  Time: 1.238s,   25.84/s  (1.288s,   24.85/s)  LR: 1.000e-04  Data: 0.013 (0.014)
INFO:root:Train: 0 [6100/40036 ( 15%)]  Loss: 70.186562 (70.4543)  Loss_l1:  0.046535 (0.1035)  Loss_traj: 69.953888 (69.9368)  Time: 1.348s,   23.74/s  (1.288s,   24.85/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [6150/40036 ( 15%)]  Loss: 70.583466 (70.4549)  Loss_l1:  0.040579 (0.1036)  Loss_traj: 70.380569 (69.9370)  Time: 1.226s,   26.10/s  (1.288s,   24.85/s)  LR: 1.000e-04  Data: 0.018 (0.014)
INFO:root:Train: 0 [6200/40036 ( 15%)]  Loss: 69.150772 (70.4555)  Loss_l1:  0.053282 (0.1038)  Loss_traj: 68.884361 (69.9366)  Time: 1.338s,   23.92/s  (1.288s,   

INFO:root:Train: 0 [7900/40036 ( 20%)]  Loss: 69.557503 (70.4635)  Loss_l1:  0.167725 (0.1036)  Loss_traj: 68.718880 (69.9456)  Time: 1.223s,   26.16/s  (1.287s,   24.86/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [7950/40036 ( 20%)]  Loss: 70.302559 (70.4619)  Loss_l1:  0.002479 (0.1036)  Loss_traj: 70.290161 (69.9439)  Time: 1.343s,   23.83/s  (1.287s,   24.86/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [8000/40036 ( 20%)]  Loss: 69.737579 (70.4620)  Loss_l1:  0.033728 (0.1037)  Loss_traj: 69.568939 (69.9436)  Time: 1.215s,   26.34/s  (1.287s,   24.86/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [8050/40036 ( 20%)]  Loss: 69.684685 (70.4609)  Loss_l1:  0.054700 (0.1036)  Loss_traj: 69.411179 (69.9428)  Time: 1.336s,   23.95/s  (1.287s,   24.86/s)  LR: 1.000e-04  Data: 0.013 (0.014)
INFO:root:Train: 0 [8100/40036 ( 20%)]  Loss: 71.305168 (70.4609)  Loss_l1:  0.057690 (0.1036)  Loss_traj: 71.016716 (69.9427)  Time: 1.315s,   24.33/s  (1.287s,   

...

INFO:root:Train: 0 [25850/40036 ( 65%)]  Loss: 69.724663 (70.4674)  Loss_l1:  0.172831 (0.1040)  Loss_traj: 68.860512 (69.9476)  Time: 1.225s,   26.13/s  (1.286s,   24.88/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [25900/40036 ( 65%)]  Loss: 71.843719 (70.4678)  Loss_l1:  0.165653 (0.1040)  Loss_traj: 71.015457 (69.9480)  Time: 1.340s,   23.88/s  (1.286s,   24.88/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [25950/40036 ( 65%)]  Loss: 68.666862 (70.4676)  Loss_l1:  0.129379 (0.1040)  Loss_traj: 68.019966 (69.9478)  Time: 1.354s,   23.64/s  (1.286s,   24.88/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [26000/40036 ( 65%)]  Loss: 69.353714 (70.4673)  Loss_l1:  0.065023 (0.1040)  Loss_traj: 69.028595 (69.9475)  Time: 1.211s,   26.43/s  (1.286s,   24.88/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [26050/40036 ( 65%)]  Loss: 68.308189 (70.4678)  Loss_l1:  0.026067 (0.1040)  Loss_traj: 68.177856 (69.9479)  Time: 1.347s,   23.75/s  (1.286

INFO:root:Train: 0 [27850/40036 ( 70%)]  Loss: 70.037170 (70.4685)  Loss_l1:  0.094765 (0.1040)  Loss_traj: 69.563347 (69.9482)  Time: 1.258s,   25.44/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.015 (0.014)
INFO:root:Train: 0 [27900/40036 ( 70%)]  Loss: 67.889305 (70.4688)  Loss_l1:  0.109444 (0.1040)  Loss_traj: 67.342087 (69.9486)  Time: 1.339s,   23.91/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.016 (0.014)
INFO:root:Train: 0 [27950/40036 ( 70%)]  Loss: 69.672432 (70.4690)  Loss_l1:  0.035997 (0.1040)  Loss_traj: 69.492447 (69.9489)  Time: 1.242s,   25.76/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.015 (0.014)
INFO:root:Train: 0 [28000/40036 ( 70%)]  Loss: 69.723480 (70.4686)  Loss_l1:  0.112788 (0.1040)  Loss_traj: 69.159538 (69.9485)  Time: 1.325s,   24.14/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.015 (0.014)
INFO:root:Train: 0 [28050/40036 ( 70%)]  Loss: 70.986496 (70.4682)  Loss_l1:  0.168144 (0.1041)  Loss_traj: 70.145775 (69.9479)  Time: 1.345s,   23.80/s  (1.286

INFO:root:Train: 0 [29850/40036 ( 75%)]  Loss: 70.723785 (70.4673)  Loss_l1:  0.030739 (0.1042)  Loss_traj: 70.570091 (69.9465)  Time: 1.247s,   25.67/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [29900/40036 ( 75%)]  Loss: 71.177231 (70.4672)  Loss_l1:  0.043150 (0.1042)  Loss_traj: 70.961479 (69.9464)  Time: 1.350s,   23.70/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [29950/40036 ( 75%)]  Loss: 70.418198 (70.4673)  Loss_l1:  0.017238 (0.1041)  Loss_traj: 70.332008 (69.9466)  Time: 1.257s,   25.45/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [30000/40036 ( 75%)]  Loss: 68.364944 (70.4675)  Loss_l1:  0.028222 (0.1041)  Loss_traj: 68.223831 (69.9468)  Time: 1.337s,   23.93/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.014 (0.014)
INFO:root:Train: 0 [30050/40036 ( 75%)]  Loss: 69.494858 (70.4679)  Loss_l1:  0.071704 (0.1042)  Loss_traj: 69.136337 (69.9470)  Time: 1.235s,   25.90/s  (1.286

INFO:root:Train: 0 [31850/40036 ( 80%)]  Loss: 68.696602 (70.4690)  Loss_l1:  0.071646 (0.1042)  Loss_traj: 68.338371 (69.9481)  Time: 1.318s,   24.29/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.015 (0.014)
INFO:root:Train: 0 [31900/40036 ( 80%)]  Loss: 69.813904 (70.4690)  Loss_l1:  0.006412 (0.1042)  Loss_traj: 69.781845 (69.9479)  Time: 1.333s,   24.01/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.015 (0.014)
INFO:root:Train: 0 [31950/40036 ( 80%)]  Loss: 69.629738 (70.4690)  Loss_l1:  0.117338 (0.1042)  Loss_traj: 69.043045 (69.9479)  Time: 1.249s,   25.63/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.015 (0.014)
INFO:root:Train: 0 [32000/40036 ( 80%)]  Loss: 70.060341 (70.4694)  Loss_l1:  0.095062 (0.1043)  Loss_traj: 69.585030 (69.9481)  Time: 1.305s,   24.52/s  (1.286s,   24.87/s)  LR: 1.000e-04  Data: 0.015 (0.014)
INFO:root:Train: 0 [32050/40036 ( 80%)]  Loss: 70.214996 (70.4694)  Loss_l1:  0.206098 (0.1043)  Loss_traj: 69.184509 (69.9480)  Time: 1.229s,   26.03/s  (1.286

KeyboardInterrupt: 

In [39]:
for batch_idx, (inputs, target) in enumerate(train_loader):
    print(inputs)
    break

tensor([[[[ 0.0056,  0.0056,  0.0056,  ...,  0.4851,  0.4337,  0.4508],
          [ 0.0056,  0.0056,  0.0056,  ...,  0.4851,  0.4337,  0.4337],
          [ 0.0056,  0.0056,  0.0056,  ...,  0.4679,  0.4679,  0.4679],
          ...,
          [ 1.4783,  1.4783,  1.4783,  ...,  2.0263,  2.0434,  2.0434],
          [ 1.4440,  1.4440,  1.4440,  ...,  2.0092,  2.0092,  2.0092],
          [ 1.4440,  1.4440,  1.4440,  ...,  1.9578,  1.9578,  1.9578]],

         [[ 1.5182,  1.5182,  1.5182,  ...,  1.7458,  1.7458,  1.7108],
          [ 1.5182,  1.5182,  1.5182,  ...,  1.7458,  1.7458,  1.7458],
          [ 1.5182,  1.5182,  1.5182,  ...,  1.7633,  1.7633,  1.7633],
          ...,
          [-1.8782, -1.8782, -1.8782,  ..., -2.0357, -2.0357, -2.0357],
          [-1.8782, -1.8782, -1.8782,  ..., -2.0357, -2.0357, -2.0357],
          [-1.8782, -1.8782, -1.8782,  ..., -2.0357, -2.0357, -2.0357]],

         [[ 2.6400,  2.6400,  2.6400,  ...,  2.6400,  2.6400,  2.6400],
          [ 2.6400,  2.6400,  

In [40]:
for batch_idx, (inputs, target) in enumerate(train_loader):
    print(inputs)
    break

tensor([[[[ 2.0263,  2.0263,  2.0263,  ...,  1.9064,  1.8379,  1.7865],
          [ 2.0263,  2.0263,  2.0263,  ...,  1.7523,  1.7180,  1.7009],
          [ 2.0263,  2.0263,  2.0263,  ...,  1.4098,  1.5125,  1.5639],
          ...,
          [-0.1486, -0.1486, -0.1314,  ..., -0.4226, -0.4568, -0.4739],
          [-0.1657, -0.1657, -0.1657,  ..., -0.3712, -0.4397, -0.4739],
          [-0.1657, -0.1657, -0.1999,  ..., -0.3541, -0.4397, -0.4739]],

         [[ 2.2010,  2.2010,  2.2010,  ...,  0.0476,  0.1001,  0.1176],
          [ 2.2010,  2.2010,  2.2010,  ...,  0.0126,  0.0826,  0.1001],
          [ 2.2010,  2.2010,  2.2010,  ..., -0.1099, -0.0049,  0.0126],
          ...,
          [ 0.0126,  0.0126,  0.0301,  ..., -0.3025, -0.3025, -0.2675],
          [ 0.0126,  0.0126,  0.0126,  ..., -0.2675, -0.3025, -0.3025],
          [ 0.0126,  0.0126, -0.0049,  ..., -0.2675, -0.3025, -0.3200]],

         [[ 2.4134,  2.4134,  2.4134,  ...,  1.4374,  1.3851,  1.3851],
          [ 2.4134,  2.4134,  

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 60, in _worker
    output = module(*input, **kwargs)
  File "/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/cutz/thesis/timm/models/efficientnet.py", line 403, in forward
    x, forward_list = self.forward_features(x)
  File "/home/cutz/thesis/timm/models/efficientnet.py", line 394, in forward_features
    x = block(x)
  File "/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/modules/container.py", line 100, in forward
    input = module(input)
  File "/home/cutz/anaconda3/lib/python3.7/site-packages/torch/nn/modules/mo